In [8]:
import xmlrpc.client
import pandas as pd

username = "juan.cano@donsson.com"
password = "1000285668"
url = "https://donsson.com"
db = "Donsson_produccion"

common = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/common")
uid = common.authenticate(db, username, password, {})
models = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/object")
# ---------------------------------------------
# 1) Buscar BOMs que tengan routing
# ---------------------------------------------
bom_ids = models.execute_kw(
    db, uid, password,
    'mrp.bom', 'search',
    [[('routing_id', '!=', False)]],  # <= Solo BOM con Routing
)

print(f"BOMs con routing encontradas: {len(bom_ids)}")

if not bom_ids:
    print("No se encontraron BOM con routing.")
    raise SystemExit()

# Leer solo product_tmpl_id de las BOM
boms = models.execute_kw(
    db, uid, password,
    'mrp.bom', 'read',
    [bom_ids],
    {'fields': ['id', 'product_tmpl_id', 'routing_id']}
)

product_template_ids = sorted({b['product_tmpl_id'][0] for b in boms})
print(f"Templates con routing encontrados: {len(product_template_ids)}")

# ---------------------------------------------
# 2) Obtener product.product a partir del template
# ---------------------------------------------
product_ids = models.execute_kw(
    db, uid, password,
    'product.product', 'search',
    [[('product_tmpl_id', 'in', product_template_ids)]],
)


# 3) Buscar Movimientos de Stock (stock.move)
#    con cantidad consumida (product_qty) mayor a 0
# ---------------------------------------------

move_ids = models.execute_kw(
    db, uid, password,
    'stock.move', 'search',
    [[
        ('product_id', 'in', product_ids),
        ('production_id', '!=', False),   # Movimientos de producto terminado
        ('product_qty', '>', 0),          # Cantidad positiva
        ('state', '=', 'done'),           # Movimientos completados
        ('date', '>=', '2024-01-01')
    ]],
    {'limit': 0}  # Asegura que no se limite el número de resultados
)
if not move_ids:
    print("No se encontraron movimientos de stock de consumo > 0 para estos productos.")
    raise SystemExit()

# Leer los movimientos con campos relevantes para el análisis de costo
stock_moves = models.execute_kw(
    db, uid, password,
    'stock.move', 'read',
    [move_ids],
    {'fields': ['id', 'product_id', 'date', 'product_qty', 'price_unit', 'production_id', 'state', 'location_id', 'location_dest_id']}
)

df_moves = pd.DataFrame(stock_moves)

print("\nDataFrame de Movimientos de Stock Filtrado (Primeras 5 filas):")


BOMs con routing encontradas: 739
Templates con routing encontrados: 738

DataFrame de Movimientos de Stock Filtrado (Primeras 5 filas):


In [55]:
df_moves.sample(31)

,product_id,price_unit,state,location_dest_id,production_id,product_qty,date,location_id,id
2723,"[16784, [DAR02032025] DA2032 FILTRO AIRE CAMIO...",25276.969812,done,"[1327, BD1/Entrada-Produccion]","[10258, MO10393]",40.0,2024-04-10 20:34:36,"[7, Ubicaciones virtuales/Producción]",2675661
1507,"[17058, [DAB02570025] DA2570 FILTRO AIRE DONSS...",18026.890172,done,"[1327, BD1/Entrada-Produccion]","[11763, MO11682]",501.0,2024-12-17 15:30:53,"[7, Ubicaciones virtuales/Producción]",3000821
1466,"[36776, [DAB19184025] DA9184A FILTRO SEGURIDAD...",20967.581064,done,"[1327, BD1/Entrada-Produccion]","[11803, MO11722]",50.0,2025-01-17 13:02:08,"[7, Ubicaciones virtuales/Producción]",3011461
2601,"[17890, [DAR08040025] DA8040 FILTRO AIRE FOTON...",36267.273163,done,"[1327, BD1/Entrada-Produccion]","[10375, MO10512]",30.0,2024-05-08 15:25:29,"[7, Ubicaciones virtuales/Producción]",2713769
2986,"[16980, [DAR02450025] DA2450 FILTRO AIRE DONSS...",56155.114514,done,"[1327, BD1/Entrada-Produccion]","[9973, MO10106]",150.0,2024-02-29 16:33:14,"[7, Ubicaciones virtuales/Producción]",2608930
1312,"[17058, [DAB02570025] DA2570 FILTRO AIRE DONSS...",14265.007850,done,"[1327, BD1/Entrada-Produccion]","[12067, MO11876]",1000.0,2025-02-08 11:55:21,"[7, Ubicaciones virtuales/Producción]",3068430
2416,"[17670, [DAB04690025] DA4690 FILTRO AIRE 2o J....",14269.789199,done,"[1327, BD1/Entrada-Produccion]","[10563, MO10701]",300.0,2024-06-12 12:30:37,"[7, Ubicaciones virtuales/Producción]",2749709
132,"[16826, [DAR02109025] DA2109 FILTRO AIRE 1_ FI...",35516.776337,done,"[1327, BD1/Entrada-Produccion]","[13472, MO13202]",120.0,2025-10-20 09:00:43,"[7, Ubicaciones virtuales/Producción]",3395494
717,"[17698, [DAB04852025] DA4852 FILTRO AIRE 2_ AG...",16218.009087,done,"[1327, BD1/Entrada-Produccion]","[12715, MO12516]",150.0,2025-06-19 12:50:33,"[7, Ubicaciones virtuales/Producción]",3245754
2369,"[49562, [DAB08113025] DA8113 FILTRO AIRE - DAF...",55021.801495,done,"[1327, BD1/Entrada-Produccion]","[10676, MO10748]",40.0,2024-06-24 09:51:24,"[7, Ubicaciones virtuales/Producción]",2767953
